In [4]:
from ipywidgets import Layout, HBox, VBox, Label, Checkbox, FloatText, Dropdown, BoundedIntText
from properties import BoltMetric, Member

is_tapped_hole_checkbox = Checkbox(value=False, description='Tapped Hole?')
engagement_length_input = HBox([
    Label(value="Thread Engagement Length:"),
    FloatText(value=10, disabled=False),
    Dropdown(options=['mm', 'cm', 'm'], layout=Layout(width="auto"))
])

# Function to update visibility based on checkbox state
def update_isTappedHole_visibility(checked):
    if checked:
        engagement_length_input.layout.display = 'flex'
    else:
        engagement_length_input.layout.display = 'none'

# Connect the checkbox to the update function
is_tapped_hole_checkbox.observe(lambda change: update_isTappedHole_visibility(change['new']), names='value')
engagement_length_input.layout.display = 'none'

bolt_info_layout = VBox([
    Label(value="Bolt Info"),
    is_tapped_hole_checkbox,
    HBox([Label(value="Nom. Size:"), Dropdown(options=BoltMetric.dimensional_props.get_entries())]),
    HBox([Label(value="Class:"), Dropdown(options=BoltMetric.class_props.get_entries())]),
    HBox([Label(value="Material:"), Dropdown(options=BoltMetric.material_props.get_entries())]),
    HBox([Label(value="Length:"), FloatText(value=10.0, disabled=False), Dropdown(options=['mm', 'cm', 'm'], layout=Layout(width="auto"))]),
    HBox([Label(value="Material in grip thickness:"), FloatText(value=10.0, disabled=False), Dropdown(options=['mm', 'cm', 'm'], layout=Layout(width="auto"))]),
    engagement_length_input,
], layout=Layout(align_items='flex-start'))



# TODO: Styling breaks down after 8 members
member_count_input = BoundedIntText(value=1, min=0, max=8, layout=Layout(width="auto"))

# Container for member rows
member_rows_container = VBox([])

# Function to create a member row
def create_member_row(member_index):
    return HBox([
        Label(value=f"{member_index + 1}:"),
        FloatText(value=5.0, description='Thickness:', min=0.1, max=100.0),
        Dropdown(options=['mm', 'cm', 'm'], value='mm', layout=Layout(width="auto")),
        Dropdown(options=Member.material_props.get_entries(), description='Material:')
    ], layout=Layout(margin='1px'))

# Function to update member rows based on count
def update_member_rows(change):
    new_count = change['new']
    current_count = len(member_rows_container.children)
    
    if new_count > current_count:
        # Add new rows
        for i in range(current_count, new_count):
            member_rows_container.children = list(member_rows_container.children) + [create_member_row(i)]
    elif new_count < current_count:
        # Remove excess rows
        member_rows_container.children = list(member_rows_container.children)[:new_count]

# Connect the count input to the update function
member_count_input.observe(update_member_rows, names='value')

# Initialize with one member
member_count_input.value = 1
update_member_rows({'new': 1})

# Scrollable container for members
scrollable_members = VBox([
    member_rows_container
], layout=Layout(
    max_height='300px',
    overflow_y='auto',
    border='1px solid #ccc',
    padding='5px'
))

member_info_layout = VBox([
    Label(value="Member Info"),
    HBox([Label(value="No. of Members:"), member_count_input]),
    scrollable_members
], layout=Layout(align_items='flex-start'))

ui = HBox([
    bolt_info_layout, 
    member_info_layout
])
ui